In [1]:
import pickle
with open("list_of_grads", "rb") as fp:   #Pickling
    list_of_grads = pickle.load(fp)

In [2]:
list_of_grads[3].shape

torch.Size([192, 768])

## Prepare

In [5]:
import numpy as np
Ps = []
Qs = []

for elem in list_of_grads[:3]:
    U, S, Vt = np.linalg.svd(elem.cpu().numpy(), full_matrices=False)
    # truncate SVD and fuse Sigma matrix
    p = np.dot(np.diag(np.sqrt(S[0:50])),Vt[0:50, :])
    q = np.dot(U[:,0:50,], np.diag(np.sqrt(S[0:50])))
    Ps.append(p)
    Qs.append(q)

In [77]:
import torch
dense3_grad = torch.load("dense3.pt")

U, S, Vt = np.linalg.svd(dense3_grad.cpu().numpy(), full_matrices=False)
Q = np.dot(np.diag(np.sqrt(S[0:50])),Vt[0:50, :])
P = np.dot(U[:,0:50,], np.diag(np.sqrt(S[0:50])))

/tmp/ipykernel_724817/2531670614.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dense3_grad = torch.load("dense3.pt")


## 1. Define Kron factors

In [47]:
m, n = Qs[0].shape
print ("m, n", m, n)
list_of_grads1 = [grad.reshape(-1) for grad in Qs]

grad_vectors = np.stack([grad.reshape(n,m, order = 'F') for grad in list_of_grads1])
k, m, n = grad_vectors.shape
print ("k, m, n", k, m, n)
e_d = np.ones((m*m, 1))
res = np.zeros(n*n) 

m, n 192 50
k, m, n 3 50 192


In [64]:
def get_factor_left(list_of_lora_factors):
    m, n = list_of_lora_factors[0].shape #cols rows in torch -> rows cols in numpy
    print ("m, n", m, n)
    list_of_grads1 = [grad.reshape(-1) for grad in list_of_lora_factors]

    grad_vectors = np.stack([grad.reshape(n,m, order = 'F') for grad in list_of_grads1])
    k, m, n = grad_vectors.shape
    print ("k, m, n", k, m, n)
    e_d = np.ones((m, 1))
    res = np.zeros(n*n) 
    V = (e_d@e_d.T)
    d_size = V.shape[0]*V.shape[1]
    for i in range(k):
        res += (grad_vectors[i].T @ V @ grad_vectors[i]).T.ravel()/d_size
    return res/k

def get_factor_right(list_of_lora_factors):
    m, n = list_of_lora_factors[0].shape #cols rows in torch -> rows cols in numpy
    print ("m, n", m, n)
    list_of_grads1 = [grad.reshape(-1) for grad in list_of_lora_factors]

    grad_vectors = np.stack([grad.reshape(n,m, order = 'F') for grad in list_of_grads1])
    k, m, n = grad_vectors.shape
    print ("k, m, n", k, m, n)
    k, m, n = grad_vectors.shape
    e_d = np.ones((n, 1))
    res = np.zeros(m*m) 
    V = (e_d@e_d.T)
    d_size = V.shape[0]*V.shape[1]
    for i in range(k):
        res += (grad_vectors[i] @ V @ grad_vectors[i].T).T.ravel()/d_size
    return res/k

In [70]:
m, n = Qs[0].shape
print ("m, n", m, n)
LT = get_factor_left(Qs).reshape(m, m, order='F')

m, n 192 50
m, n 192 50
k, m, n 3 50 192


In [71]:
m, n = Ps[0].shape
print ("m, n", m, n)
R = get_factor_right(Ps).reshape(n, n, order='F')

m, n 50 768
m, n 50 768
k, m, n 3 768 50


## 2. Find 

$L^T P Q^T R \approx U \Sigma V^T $

In [78]:
LT.shape

(192, 192)

In [83]:
Q.shape

(50, 768)

In [82]:
(LT@P).shape

(192, 50)

In [84]:
U, S, Vt = np.linalg.svd(LT@P@Q@R)

# 3. Define

$\hat{Vt} = Vt R^{-1}$

$\hat{U} = L^{-T} U$

In [85]:
Vt_new = Vt @ np.linalg.inv(R)
U_new = np.linalg.inv(LT) @ U

In [87]:
Vt_new.shape

(768, 768)

In [97]:
U_new[:,:len(S)].shape

(192, 192)

In [93]:
np.diag(S).shape

(192, 192)

In [90]:
U.shape

(192, 192)

In [92]:
Vt.shape

(768, 768)

In [98]:
dW = U_new@np.diag(S)@Vt_new[:len(S)] # new delta W instead of delta W = P@Q